<a href="https://colab.research.google.com/github/someshsingh22/TeamEleven/blob/main/BaselinesFlippedFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 4.2MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


In [90]:
# Access data store
data_store = pd.HDFStore('processed_data.h5')

# Retrieve data using key
match_df = data_store['preprocessed_df']
data_store.close()

In [91]:
match_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN


In [92]:
match_df['Winner_team'] = match_df['Winner']

In [93]:
for ind in match_df.index: 
  if(match_df['Winner'][ind]==match_df['Team_1'][ind]):
    match_df['Winner_team'][ind] = 1
  elif(match_df['Winner'][ind]==match_df['Team_2'][ind]):
    match_df['Winner_team'][ind] = 2
  else:
    match_df['Winner_team'][ind] = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [94]:
match_df['Winner_team'].value_counts()

1    1967
2    1689
Name: Winner_team, dtype: int64

In [95]:
match_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2


In [96]:
np.random.seed(60)

##Calculating Net Run Rate

###Import Data

In [97]:
attributes = pd.read_csv("attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [98]:
scorecard = open('scorecard.json',) 
scorecard_data = json.load(scorecard)

In [99]:
tmap = open('tmap.json',) 
tmap_data = json.load(tmap)

###Get NNR

In [100]:
match = match_df.copy()

In [101]:
match['NRR_team1']=''
match['NRR_team2']=''

In [102]:
name_to_code = {
    
    "Afghanistan" : "AFG",
    "Australia" : "AUS",
    "Bangladesh" : "BAN",
    "England" : "ENG",
    "India" : "IND",
    "Ireland" : "IRE",
    "Kenya" : "KEN",
    "Netherlands" : "NED",
    "New Zealand" : "NZL",
    "Pakistan" : "PAK",
    "South Africa" : "SAF",
    "Scotland" : "SCO",
    "Sri Lanka" : "SRL",
    "West Indies" : "WIN",
    "Zimbabwe" : "ZIM"

}

In [111]:
skip_keys = ['1282',
'1761',
'1765',
'1770',
'1862',
'1866',
'2528'
]

In [103]:
def check_allOut(scorecard_data, matchCode, team_num):
    
    bat = 'BATTING' + str(team_num)
    
    dismissal = [i[1] for i in scorecard_data[matchCode][bat]]
    
    if("not out" in dismissal or '' in dismissal):
        return False
    
    return True

In [104]:
def get_totalOvers(scorecard_data, matchCode, team_num):
    
    bat = 'BATTING' + str(team_num)
    
    balls = [i[3] for i in scorecard_data[matchCode][bat] if i[3]!=-1 ]
    
    overs = sum(balls)/6
    return overs

In [112]:
for ind in match.index:
  if (match['Winner_team'][ind]==0):
    match['NRR_team1'][ind] = 0
    match['NNR_team2'][ind] = 0
  else:
    team_num = 2
    match_code = str(match['MatchCode'][ind])
    if (match_code in skip_keys):
      continue
    order=scorecard_data[match_code]['ORDER']
    if (name_to_code[order[1]]==match['Team_2'][ind]):
      team_num = 1
    runRate_team1=match['Score_1'][ind]/50
    if (check_allOut(scorecard_data,match_code,team_num)):
      runRate_team2=match['Score_2'][ind]/50
    else:
      if(match['Winner_team'][ind]==2):
        runRate_team2=match['Score_2'][ind]/get_totalOvers(scorecard_data, match_code, team_num)
      else:
        runRate_team2=match['Score_2'][ind]/50

    match['NRR_team1'][ind]= runRate_team1 - runRate_team2
    match['NRR_team2'][ind]= runRate_team2 - runRate_team1



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [113]:
match.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1,0.64,-0.64
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2,-0.14,0.14


In [114]:
len(match)

3656

In [118]:
match = match[~match['MatchCode'].isin(skip_keys)]

In [119]:
len(match)

3649

###Store the NNR dataframe

In [120]:
# Save the final dataframe
data_store = pd.HDFStore('nnr_data.h5')

data_store['preprocessed_df'] = match
data_store.close()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Date', 'Team_1', 'Team_2', 'TOSS', 'Winner', 'Winner_team',
       'NRR_team1', 'NRR_team2'],
      dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


#Flipped Dataset

In [121]:
# Access data store
data_store = pd.HDFStore('nnr_data.h5')

# Retrieve data using key
match_df = data_store['preprocessed_df']
data_store.close()

In [122]:
match_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1,0.64,-0.64
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2,-0.14,0.14


In [125]:
match_flipped_df = match_df.copy()

In [130]:
for ind in match_flipped_df.index: 
  match_flipped_df['Team_1'][ind], match_flipped_df['Team_2'][ind] = match_df['Team_2'][ind],match_df['Team_1'][ind]
  match_flipped_df['Score_1'][ind], match_flipped_df['Score_2'][ind] = match_df['Score_2'][ind],match_df['Score_1'][ind]
  match_flipped_df['NRR_team1'][ind], match_flipped_df['NRR_team2'][ind] = match_df['NRR_team2'][ind],match_df['NRR_team1'][ind]
  

  if (match_df['TOSS'][ind]==1):
    match_flipped_df['TOSS'][ind] = 2
  else:
    match_flipped_df['TOSS'][ind] = 1

  if (match_df['Venue'][ind]==1):
    match_flipped_df['Venue'][ind] = 2
  elif (match_df['Venue'][ind]==2):
    match_flipped_df['Venue'][ind] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [131]:
for ind in match_flipped_df.index: 
  if(match_flipped_df['Winner'][ind]==match_flipped_df['Team_1'][ind]):
    match_flipped_df['Winner_team'][ind] = 1
  else:
    match_flipped_df['Winner_team'][ind] = 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [132]:
match_flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AUS,AFG,2,3773,65,2,417,142,AUS,1,5.5,-5.5
1,2012-08-25,AUS,AFG,1,3437,156,1,272,206,AUS,1,1.32,-1.32
2,2019-06-01,AUS,AFG,1,4306,149,2,209,207,AUS,1,0.04,-0.04
3,2014-03-01,BAN,AFG,2,3623,190,1,222,254,AFG,2,-0.64,0.64
4,2016-09-25,BAN,AFG,2,3936,198,1,265,258,BAN,1,0.14,-0.14


In [ ]:
# Access data store
data_store = pd.HDFStore('processed_data.h5')

# Retrieve data using key
match_df = data_store['preprocessed_df']
data_store.close()

In [134]:
frames = [match_df, match_flipped_df]

final_df = pd.concat(frames)

In [135]:
final_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1,0.64,-0.64
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2,-0.14,0.14


In [136]:
len(final_df)

7298

In [137]:
# Save the final dataframe
data_store = pd.HDFStore('flipped_data.h5')

data_store['flip_df'] = final_df
data_store.close()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Date', 'Team_1', 'Team_2', 'TOSS', 'Winner', 'Winner_team',
       'NRR_team1', 'NRR_team2'],
      dtype='object')]

  exec(code_obj, self.user_global_ns, self.user_ns)


####Flipped dataframe

In [138]:
# Access data store
data_store = pd.HDFStore('flipped_data.h5')

# Retrieve data using key
flipped_df = data_store['flip_df']
data_store.close()

In [139]:
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,AFG,1,0.64,-0.64
4,2016-09-25,AFG,BAN,1,3936,198,2,258,265,BAN,2,-0.14,0.14


In [140]:
len(flipped_df)

7298

#Models using Flipped Data

In [141]:
enc = OneHotEncoder(handle_unknown='ignore')

In [142]:
enc_df = pd.DataFrame(enc.fit_transform(flipped_df[['Winner_team','TOSS']]).toarray())

In [143]:
flipped_df = flipped_df.join(enc_df)
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2,0,1,2,3
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5,0.0,1.0,1.0,0.0
0,2015-03-04,AUS,AFG,2,3773,65,2,417,142,AUS,1,5.5,-5.5,0.0,1.0,1.0,0.0
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32,0.0,1.0,0.0,1.0
1,2012-08-25,AUS,AFG,1,3437,156,1,272,206,AUS,1,1.32,-1.32,0.0,1.0,0.0,1.0
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04,0.0,1.0,1.0,0.0


In [144]:
flipped_df.rename(columns = {0: 'Win1',1: 'Win2',2: 'Toss1',3: 'Toss2'}, inplace = True)

In [145]:
labelencoder = LabelEncoder()

flipped_df['Team_1Enc'] = labelencoder.fit_transform(flipped_df['Team_1'])
flipped_df['Team_2Enc'] = labelencoder.fit_transform(flipped_df['Team_2'])

In [146]:
flipped_df.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,TOSS,Score_1,Score_2,Winner,Winner_team,NRR_team1,NRR_team2,Win1,Win2,Toss1,Toss2,Team_1Enc,Team_2Enc
0,2015-03-04,AFG,AUS,1,3773,65,1,142,417,AUS,2,-5.5,5.5,0.0,1.0,1.0,0.0,0,1
0,2015-03-04,AUS,AFG,2,3773,65,2,417,142,AUS,1,5.5,-5.5,0.0,1.0,1.0,0.0,1,0
1,2012-08-25,AFG,AUS,2,3437,156,2,206,272,AUS,2,-1.32,1.32,0.0,1.0,0.0,1.0,0,1
1,2012-08-25,AUS,AFG,1,3437,156,1,272,206,AUS,1,1.32,-1.32,0.0,1.0,0.0,1.0,1,0
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,AUS,2,-0.04,0.04,0.0,1.0,1.0,0.0,0,1


In [166]:
X = flipped_df[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','TOSS','Toss1','Toss2']].copy()
y = flipped_df[['Winner_team','Win1','Win2','Score_1','Score_2','NRR_team1','NRR_team2']].copy()

In [167]:
# Test Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/5, random_state = 100)

In [168]:
X_train.head()

,Date,Team_1Enc,Team_2Enc,Venue,GroundCode,TOSS,Toss1,Toss2
1621,2020-02-09,3,10,1,66,1,1.0,0.0
1054,2010-07-20,2,7,2,203,2,0.0,1.0
2703,2018-11-07,8,9,2,192,1,1.0,0.0
1505,2001-06-17,3,9,0,27,2,0.0,1.0
1716,2007-07-04,13,3,0,19,2,1.0,0.0


In [169]:
y_train.head()

,Winner_team,Win1,Win2,Score_1,Score_2,NRR_team1,NRR_team2
1621,1,1.0,0.0,257,256,0.02,-0.02
1054,2,0.0,1.0,199,200,-2.64983,2.64983
2703,1,1.0,0.0,266,219,0.94,-0.94
1505,1,1.0,0.0,156,153,0.06,-0.06
1716,1,0.0,1.0,278,217,2.02641,-2.02641


##Training on Scores

In [170]:
def print_model_scores(model, data, y, predictors, target):
    '''
    A generic function to generate the performance report of the
    model in question on the data passed to it
    Args:
        model: ML Model to be checked
        data: data on which the model needs to pe trained
        y: data containing the target variables
        predictors: independent feature variable
        target: target variable
    '''
    model.fit(data[predictors], y[target])
    predictions = model.predict(data[predictors])
    rms = sklearn.metrics.mean_squared_error(predictions,y[target])**0.5
    print('RMS : %s' % '{0:.2%}'.format(rms))
    r2 = sklearn.metrics.r2_score(predictions,y[target])
    print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [171]:
def winner_prediction(model, data, y, predictors,winner):
  '''
    A generic function to predict the winner for the model in question 
    
    Args:
        model: ML Model to be checked
        data: data on which the model needs to be trained
        y: data containing the target variables
        predictors: independent feature variable
        winner: winning team
  '''
  pred1 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_1'])
  pred2 = print_model_scores(model, X_train, y_train,predictor_var, ['Score_2'])

  pred = pred1 - pred2

  for i in range(len(pred)):
    if ((pred[i])>0):
      pred[i] = 1
    else:
      pred[i] = 2

  print("Model Accuracy is: ")
  print(sum(1 for x,y in zip(pred,winner) if x == y) / len(winner))

##Model1 - Toss + GroundCode 

In [172]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 3952.37%
R2 : -5.16%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 3960.07%
R2 : -6.83%
Model Accuracy is: 
0.7261048304213772


In [173]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 6080.32%
R2 : -23188.61%
RMS : 6064.36%
R2 : -20429.76%
Model Accuracy is: 
0.5193559438163755


In [174]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 3632.85%
R2 : 44.85%
RMS : 3644.87%
R2 : 43.87%
Model Accuracy is: 
0.8189448441247003


In [175]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 6082.79%
R2 : -35965.70%


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMS : 6065.55%
R2 : -25259.06%
Model Accuracy is: 
0.5320315176430285


##Model2 - Toss + GroundCode + Venue

In [176]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 3948.65%
R2 : -5.18%


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 3953.59%
R2 : -6.52%
Model Accuracy is: 
0.7303871188763275


In [177]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 6080.02%
R2 : -22666.18%
RMS : 6063.46%
R2 : -19240.89%
Model Accuracy is: 
0.5196985268927715


In [178]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 3632.66%
R2 : 44.86%
RMS : 3644.69%
R2 : 43.88%
Model Accuracy is: 
0.8192874272010963


In [179]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMS : 6082.79%
R2 : -35965.70%
RMS : 6063.51%
R2 : -20291.76%
Model Accuracy is: 
0.5275779376498801


##Training on NRR

In [180]:
def print_model_scores(model, data, y, predictors, target):
    '''
    A generic function to generate the performance report of the
    model in question on the data passed to it using cross-validation
    
    Args:
        model: ML Model to be checked
        data: data on which the model needs to pe trained
        predictors: independent feature variable
        target: target variable
    '''
    model.fit(data[predictors], y[target])
    predictions = model.predict(data[predictors])
    #scores = cross_val_score(model, data[predictors], y[target], scoring="neg_mean_squared_error", cv=5)
    #print('Cross-Validation Score :{}'.format(np.sqrt(-scores)))
    rms = sklearn.metrics.mean_squared_error(predictions,y[target])**0.5
    print('RMS : %s' % '{0:.2%}'.format(rms))
    #print(f"Average RMSE: {np.sqrt(-scores).mean()}")
    r2 = sklearn.metrics.r2_score(predictions,y[target])
    print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [181]:
def winner_prediction(model, data, y, predictors,winner):
    pred = print_model_scores(model, X_train, y_train,predictor_var, ['NRR_team1'])
    
    
    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = 1
        elif (pred[i]==0):
            pred[i] = 0
        else:
            pred[i] = 2

    print("Model Accuracy is: ")
    print(sum(1 for x,y in zip(pred,winner) if x == y) / len(winner))

##Model1 - Toss + GroundCode 

In [182]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 273.58%
R2 : -91.20%
Model Accuracy is: 
0.7226789996574169


In [183]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 360.78%
R2 : -26519.84%
Model Accuracy is: 
0.4725933538883179


In [184]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 261.69%
R2 : -10.15%
Model Accuracy is: 
0.7959917780061665


In [185]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 360.78%
R2 : -26689.92%
Model Accuracy is: 
0.4725933538883179


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##Model2 - Toss + GroundCode + Venue

In [186]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


RMS : 273.17%
R2 : -89.29%
Model Accuracy is: 
0.7273038711887633


In [187]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 360.76%
R2 : -25795.41%
Model Accuracy is: 
0.47704693388146624


In [188]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 261.68%
R2 : -10.14%
Model Accuracy is: 
0.7963343610825625


In [189]:
winner = y_train['Winner_team']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','TOSS','Venue']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 360.88%
R2 : -37471.64%
Model Accuracy is: 
0.4681397738951696


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
